In [ ]:
#imports 
import json
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix as get_dm
from numba import njit, prange

In [ ]:
# import JSON data

with open("2021_05_25_apis/allEndpoints.json","r",encoding="utf-8") as json_file:
     apis = json.load(json_file)
        

In [ ]:
print("Number of APIs: ",len(apis))
#calculate number of endpoints
endpoint_lens = [len(apis[i]["endpoints"]) for i in range(len(apis))]
print("Total number of Endpoints: ",sum(endpoint_lens))
print("AVG number of Endpoints: ", sum(endpoint_lens)/len(apis))

## Define Datastructures

In [ ]:
import operator


def string_to_list(value):
        s_list = list(value)
        out_list = []
        for i in range(len(s_list)):
            if operator.contains('!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',s_list[i]):
                out_list.append('.')
            else:
                if s_list[i].isupper():
                    out_list.append('.')
                    out_list.append(s_list[i].lower())
                else:
                    out_list.append(s_list[i])

        out_string = "".join(out_list)
        return [x for x in out_string.split('.') if x]
    

class Api:
    def __init__(self,api_data):
        self._raw_data = api_data
        self.key = api_data["key"]
        self.name = api_data["name"]
        self.version_key = api_data["versionKey"]
        self.version_name = api_data["versionName"]
        self.endpoints = [Endpoint(api_data["endpoints"][i],i) for i in range(len(api_data["endpoints"]))]
    
    def get_property(self,name):
        return self._raw_data[name]
    
    def has_endpoints(self):
        return len(self.endpoints) != 0
    
    def __str__(self):
        json_dict = {}
        json_dict["name"] = self.name
        json_dict["key"] = self.key
        json_dict["version_name"] = self.version_name
        json_dict["version_key"] = self.version_key
        json_dict["endpoints_size"] = len(self.endpoints)
        return json.dumps(json_dict)
    
        
    
    
        
class Endpoint:
    def __init__(self,endpoint_data,endpoint_num):
        self._raw_data = endpoint_data
        self.path = endpoint_data["path"]
        self.method = endpoint_data["method"]
        self.request_parameters = [Parameter(endpoint_data["requestParameters"][i]) for i in range(len(endpoint_data["requestParameters"]))]
        self.response_parameters = [Parameter(endpoint_data["responseParameters"][i]) for i in range(len(endpoint_data["responseParameters"]))]
        self.path_list = self.path_to_list()
        self.num = endpoint_num
    
    def get_property(self,name):
        return self._raw_data[name]
    
    def has_parameters(self):
        return (len(self.request_parameters) != 0 or len(self.response_parameters) != 0)
    
    def path_to_list(self):
        return string_to_list(self.path)
    
    def __str__(self):
        json_dict = {}
        json_dict["method"] = self.method
        json_dict["path"] = self.path
        json_dict["request_parameters_size"] = len(self.request_parameters)
        json_dict["response_parameters_size"] = len(self.response_parameters)
        return json.dumps(json_dict)
    
class Parameter:
    def __init__(self,parameter_data):
        self._raw_data = parameter_data
        self.xpath = parameter_data["xpath"]
        self.name = parameter_data["name"]
        
        #if xpath is empty, overwrite it with name
        if not self.xpath:
            self.xpath = self.name
            
        self.xpath_list = self.xpath_to_list()
        self.name_list = self.name_to_list()
    
    def get_property(self,name):
        return self._raw_data[name]
    
    def name_to_list(self):
        return string_to_list(self.name)
        
    def xpath_to_list(self):
        return string_to_list(self.xpath)
    
    def __str__(self):
        json_dict = {}
        json_dict["name"] = self.name
        json_dict["xpath"] = self.xpath
        json_dict["name_list"] = self.name_list
        json_dict["xpath_list"] = self.xpath_list
        return json.dumps(json_dict)
    
    
        

## Load Data

In [ ]:
apis_list = [Api(apis[i]) for i in range(len(apis))]

In [ ]:
# Test: print data for one api
api = 2

#Some tests:
print("API:")
print(apis_list[api])
print()
print("Endpoints:")

for endpoint in apis_list[api].endpoints:
    print("Endpoint:")
    print(endpoint)
    print()
    print("request parameters:")
    for rp in endpoint.request_parameters:
        print(rp)
    print()
    print("response parameters:")
    for rp in endpoint.response_parameters:
        print(rp)
    print()
    print()
    
    

## Load Embeddings

In [ ]:
# functions to load embeddings from file

def load_embedding_from_json(file_path:str):    
    with open(file_path,"r",encoding="utf-8") as json_file:
         word_embedding = json.load(json_file)       
    # convert vectors from list to np array
    for key, vector in word_embedding.items():
        word_embedding[key] = np.array(vector)
    return word_embedding

def load_glove_embedding_from_file(file_path):
    df = pd.read_csv(file_path, sep=" ", quoting=3, header=None, index_col=0)
    glove = {key: val.values for key, val in df.T.items()}
    return glove

In [ ]:
# load open api embedding 
api_embedding = load_embedding_from_json("saved_embeddings/open_api_embedding_5d_314_words.json")

In [ ]:
#load glove embedding
glove_embedding = load_glove_embedding_from_file("saved_embeddings/glove.6B.50d.txt")

### Reduce Glove Embedding to same Vocabulary as OpenAPI Embedding

In [ ]:
def extract_words_and_vectors_from_embedding(embedding: dict):
    words = []
    vectors = []
    
    for word, vector in embedding.items():
        words.append(word)
        vectors.append(vector)
    
    return words, np.array(vectors)

In [ ]:
api_embedding_words, api_embedding_vectors = extract_words_and_vectors_from_embedding(api_embedding)

In [ ]:
def reduce_embedding_to_words(embedding: dict, words: list):
    reduced_embedding = {}
    for w in words:
        try:
            vector = embedding[w]
            reduced_embedding[w] = vector
        except:
            pass
    return reduced_embedding    

In [ ]:
# reduce glove embedding to the same words as api embedding
glove_embedding = reduce_embedding_to_words(glove_embedding, api_embedding_words)

In [ ]:
print("Size own embedding: "+str(len(api_embedding)))
print("Size glove embedding: "+str(len(glove_embedding)))

## Define Datastructure for Requests 

In [ ]:
class Request:
    def __init__(self, endpoint_name:str, method:str, parameters:list):
        self.endpoint = endpoint_name
        self.method = method
        self.parameters = parameters
        
    def __str__(self):
        json_dict = {}
        json_dict["endpoint"] = self.endpoint
        json_dict["method"] = self.method
        json_dict["parameters"] = self.parameters
        return json.dumps(json_dict)

### Create Request Object for requests that are completely embedded 

In [ ]:


def create_request_obj_if_embedded(embedding:dict, endpoint, min_words:int):
    is_suitable = True
    parameter_words = []
    for request_parameter in endpoint.request_parameters:
        for word in request_parameter.xpath_list:
            if word in embedding:
                parameter_words.append(word)
            else:
                is_suitable = False
                break
        if not is_suitable:
            break
    
    parameter_words = list(set(parameter_words))
    
    if len(parameter_words) < min_words:
        is_suitable = False        
    
    if is_suitable:
        request_obj = Request(endpoint.path[1:], endpoint.method, parameter_words)
    else:
        request_obj = None
        
    return is_suitable, request_obj

In [ ]:
requests = []

for api in apis_list:
    for endpoint in api.endpoints:
        is_embedded, request_obj = create_request_obj_if_embedded(glove_embedding, endpoint, min_words=4)
        
        if is_embedded:
            requests.append(request_obj)
        
len(requests)

In [ ]:
print(requests[5])

In [ ]:
from functools import cmp_to_key
def compare(item1, item2):
    if len(item1.parameters) < len(item2.parameters):
        return 1
    elif len(item1.parameters) > len(item2.parameters):
        return -1
    else:
        return 0
    
requests.sort(key=cmp_to_key(compare))

In [ ]:
print(requests[0])

# Predict next words by Distance in Vector Space

In [ ]:
from scipy.spatial import distance
from functools import cmp_to_key

def get_next_words(known_words:list, embedding: dict):
    def get_wordlist_distance_to_point(center_point, embedding:dict, known_words:list):
        words = [] # list of elements (word, distance)
        for word, vector in embedding.items():
            if word not in known_words:
                words.append((distance.euclidean(center_point, vector), word))
        return words
            
    def compare(item):
            return item[0]
    
    center_point = embedding[known_words[0]]
    for i in range(1, len(known_words)):
        center_point = np.add(center_point, embedding[known_words[i]])
    center_point = center_point/len(known_words)
    
    words = get_wordlist_distance_to_point(center_point, embedding, known_words)    
    list.sort(words, key=compare)
    
    return words 

In [ ]:
def get_prediction_result(request, number_known_words:int, embedding:dict):
    known_words = request.parameters[:number_known_words]
    words_to_predict = request.parameters[number_known_words:]
    number_words_in_request = len(request.parameters)
    
    prediction_order = [i[1] for i in get_next_words(known_words, embedding)]
    
    prediction_delay = [max(0, prediction_order.index(w)-len(words_to_predict)+1) for w in words_to_predict]
    
    number_correct = 0
    for i in prediction_delay:
        if i==0:
            number_correct += 1
            
    
    return number_correct/len(prediction_delay), sum(prediction_delay)/len(prediction_delay)

In [ ]:
import random

req = requests[1]
num_param_in_reqest = len(req.parameters)
known = random.randint(1, num_param_in_reqest-1)

portion_correct_api, avg_delay_api_embedding = get_prediction_result(req, known, api_embedding)
portion_correct_glove, avg_delay_glove_embedding = get_prediction_result(req, known, glove_embedding)

print("portion correct own embedding: "+str(portion_correct_api))
print("avg delay own embedding: "+str(avg_delay_api_embedding))
print()
print("portion correct glove embedding: "+str(portion_correct_glove))
print("avg delay glove embedding: "+str(avg_delay_glove_embedding))
print()
print("known: "+str(known))
print("num_param_in_reqest: "+str(num_param_in_reqest))
print()
print("known parameters: "+str(req.parameters[:known]))
print("parameters to predict : "+str(req.parameters[known:]))

In [ ]:
import math

error_own_embedding = []
error_glove_embedding = []

for request in requests:
    known = math.floor(len(request.parameters)/2)
    portion_correct_api, avg_delay_api_embedding = get_prediction_result(request, known, api_embedding)
    portion_correct_glove, avg_delay_glove_embedding = get_prediction_result(request, known, glove_embedding)
    
    error_own_embedding.append(avg_delay_api_embedding)
    error_glove_embedding.append(avg_delay_glove_embedding)

    

In [ ]:
print("avg delay own embedding: "+str(sum(error_own_embedding)/len(error_own_embedding)))
print("avg delay glove embedding: "+str(sum(error_glove_embedding)/len(error_glove_embedding)))

# Prediction with NN 

## Reduce Glove to 5 Dimensions with PCA

In [ ]:
glove_embedding_words, glove_embedding_vectors = extract_words_and_vectors_from_embedding(glove_embedding)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 5)
glove_embedding_vectors = pca.fit_transform(glove_embedding_vectors)

In [ ]:
def create_embedding_dict(embedding_words, embedding_vectors):
    embedding = {}
    
    for i in range(len(embedding_words)):
        embedding[embedding_words[i]] = embedding_vectors[i]
        
    return embedding

In [ ]:
glove_embedding = create_embedding_dict(glove_embedding_words, glove_embedding_vectors)

## Standardize Embedding

1. Center the embedding arround the point of origin (sum of all vectors shall be zero)
2. skale all Vectors linearily with a factor that the largest absolute value of any component of any vector is 1. <br>
   --> every Component in every vector is in the range [-1;1]

In [ ]:
def standardize_embedding(embedding: dict):
    def extract_words_and_vectors_from_embedding(embedding: dict):
        words = []
        vectors = []
        
        for word, vector in embedding.items():
            words.append(word)
            vectors.append(vector)
        
        return words, np.array(vectors)

    def relocate_vectors_to_center(vectors):
        center = []
        for i in range(len(vectors[0])):
            center.append(np.sum(vectors[:,i]))
        center = np.array(center)/len(vectors)
        vectors -= center
        return vectors 
    
    words, vectors = extract_words_and_vectors_from_embedding(embedding)

    vectors = relocate_vectors_to_center(vectors)

    # scale vectors
    scalar = 1/max(abs(np.amin(vectors)), np.amax(vectors))
    print("scalar = "+str(scalar))

    for i in range(len(vectors)):
        vectors[i] = vectors[i]*scalar
    
    # update embedding

    for i, word in enumerate(words):
        embedding[word] = vectors[i]
    
    return embedding

In [ ]:
api_embedding = standardize_embedding(api_embedding)
glove_embedding = standardize_embedding(glove_embedding)

## Define Forbidden Input Words

Define 3 forbidden Input Words that occur at least once all in the same request:
1. Pick random Request
2. Pick 3 random words from that request

In [ ]:
def get_forbidden_words():
    # take random request
    param_to_choose_from = random.choice(requests).parameters

    forbidden_words = []
    while len(forbidden_words) < 3:
        w = random.choice(param_to_choose_from)
        if w not in forbidden_words:
            forbidden_words.append(w)

    return forbidden_words

In [ ]:
forbidden_words = get_forbidden_words()
forbidden_words

## Map Word to Index in One-Hot Vector

In [ ]:
def get_word_to_index_dict(wordlist:list):
    res = {}
    for i, word in enumerate(wordlist):
        res[word] = i
    return res

In [ ]:
# make word -> index dict
index_of_word_glove = get_word_to_index_dict(glove_embedding_words)
index_of_word_api = get_word_to_index_dict(api_embedding_words)

## Create Training and Test Data

In [ ]:
# all possible subsets of size k
def get_all_possible_subsets_of_length_k_by_binomial_coefficient(number_to_choose_k:int, size_of_list_to_choose_from_n:int):
    '''returns all combinations of (number_to_choose_k) indices in List with size (size_of_list_to_choose_from_n)'''
    def get_next_combination(current_combination):
        def move_later_indices_to_min_value(index_in_combination_just_set:int, current_combination):
            value_just_set = current_combination[index_in_combination_just_set]
            for i in range(1, len(current_combination) - index_in_combination_just_set):
                current_combination[index_in_combination_just_set+i] = value_just_set+i
            return current_combination
        
        for pos in range(len(current_combination)-1, -1, -1): # iterate backwards through list
            pos_value = current_combination[pos]
            if pos_value+1 < size_of_list_to_choose_from_n and pos_value+1 not in current_combination:
                current_combination[pos] += 1
                current_combination = move_later_indices_to_min_value(pos, current_combination)
                break
        return current_combination

    sets = []
    combination = [i for i in range(number_to_choose_k)]
    last_combination = [i for i in range(size_of_list_to_choose_from_n-number_to_choose_k, size_of_list_to_choose_from_n)]

    sets.append(combination.copy())
    while combination != last_combination:
        combination = get_next_combination(combination)
        sets.append(combination.copy())
    return np.array(sets)

In [ ]:
get_all_possible_subsets_of_length_k_by_binomial_coefficient(3, 5)

In [ ]:
# get_data_and_wanted results from requests

def get_data_for_training(requests, embedding:dict, index_of_word_dict: dict, forbidden_words:list):
    def get_one_hot_output(words:list, index_of_word_dict: dict, size:int):
        one_hot = np.zeros(size, dtype=np.float32)
        for w in words:
            one_hot[index_of_word_dict[w]] = 1
        return one_hot

    def get_input(words:list, embedding:dict):
        input = []
        for i in range(3):
            try:
                vector = embedding[words[i]]
            except:
                vector = np.zeros(5)
            
            input.append(vector)

        # result are all permutations
        result = np.array([np.concatenate((input[0],input[1],input[2])),
                           np.concatenate((input[0],input[2],input[1])),
                           np.concatenate((input[1],input[0],input[2])),
                           np.concatenate((input[1],input[2],input[1])),
                           np.concatenate((input[2],input[0],input[1])),
                           np.concatenate((input[2],input[1],input[0]))])
        return result


    input_data_zero_forbidden_words = []
    labels_zero_forbidden_words = []

    input_data_one_forbidden_words = []
    labels_one_forbidden_words = []

    input_data_two_forbidden_words = []
    labels_two_forbidden_words = []

    input_data_three_forbidden_words = []
    labels_three_forbidden_words = []


    size_output = len(embedding)

    for req in requests:
        param = req.parameters

        index_variants_known_words = get_all_possible_subsets_of_length_k_by_binomial_coefficient(3, len(param))

        for indexes in index_variants_known_words:
            w0 = param[indexes[0]]
            w1 = param[indexes[1]]
            w2 = param[indexes[2]]
            words = [w0, w1, w2]

            new_inputs = get_input(words, embedding)

            # determine number forbidden words

            num_forbidden_words_in_input = 0

            for w in words:
                if w in forbidden_words:
                    num_forbidden_words_in_input += 1
            
            # 0 forbidden word in input
            input_list = input_data_zero_forbidden_words
            labels_list = labels_zero_forbidden_words

            if num_forbidden_words_in_input == 1:
                input_list = input_data_one_forbidden_words
                labels_list = labels_one_forbidden_words

            elif num_forbidden_words_in_input == 2:
                input_list = input_data_two_forbidden_words
                labels_list = labels_two_forbidden_words

            elif num_forbidden_words_in_input == 3:
                input_list = input_data_three_forbidden_words
                labels_list = labels_three_forbidden_words
            
            
            # len(new_inputs) should be 6 (permutation of 3 input embeddings)

            words_to_predict = param.copy()
            words_to_predict.remove(w0)
            words_to_predict.remove(w1)
            words_to_predict.remove(w2)

            wanted_one_hot_output = get_one_hot_output(words_to_predict, index_of_word_dict, size_output)

            for ni in new_inputs:
                input_list.append(ni)
                labels_list.append(wanted_one_hot_output)

    return (np.array(input_data_zero_forbidden_words), np.array(labels_zero_forbidden_words),
           np.array(input_data_one_forbidden_words), np.array(labels_one_forbidden_words),
           np.array(input_data_two_forbidden_words), np.array(labels_two_forbidden_words),
           np.array(input_data_three_forbidden_words), np.array(labels_three_forbidden_words))
    

In [ ]:
# training / test data api embedding
input_zero_forbidden_api, labels_zero_forbidden_api, input_one_forbidden_api, labels_one_forbidden_api, input_two_forbidden_api, labels_two_forbidden_api, input_three_forbidden_api, labels_three_forbidden_api = get_data_for_training(requests, api_embedding, index_of_word_api, forbidden_words)

print("Data 0 forbidden input words")
print(input_zero_forbidden_api.shape)
print(labels_zero_forbidden_api.shape)
print()
print("Data 1 forbidden input word")
print(input_one_forbidden_api.shape)
print(labels_one_forbidden_api.shape)
print()
print("Data 2 forbidden input words")
print(input_two_forbidden_api.shape)
print(labels_two_forbidden_api.shape)
print()
print("Data 3 forbidden input words")
print(input_three_forbidden_api.shape)
print(labels_three_forbidden_api.shape)
print()


In [ ]:
# training / test data api embedding
input_zero_forbidden_glove, labels_zero_forbidden_glove, input_one_forbidden_glove, labels_one_forbidden_glove, input_two_forbidden_glove, labels_two_forbidden_glove, input_three_forbidden_glove, labels_three_forbidden_glove = get_data_for_training(requests, glove_embedding, index_of_word_glove, forbidden_words)

print("Data 0 forbidden input words")
print(input_zero_forbidden_glove.shape)
print(labels_zero_forbidden_glove.shape)
print()
print("Data 1 forbidden input word")
print(input_one_forbidden_glove.shape)
print(labels_one_forbidden_glove.shape)
print()
print("Data 2 forbidden input words")
print(input_two_forbidden_glove.shape)
print(labels_two_forbidden_glove.shape)
print()
print("Data 3 forbidden input words")
print(input_three_forbidden_glove.shape)
print(labels_three_forbidden_glove.shape)
print()

In [ ]:
random_seed = 42  # guarantees that input and labels are shuffeled in the same way
def shuffle_arrays(input_data, labels):
    np.random.seed(random_seed)
    np.random.shuffle(input_data)

    np.random.seed(random_seed)
    np.random.shuffle(labels)

    return input_data, labels

In [ ]:
# shuffle api data
input_zero_forbidden_api, labels_zero_forbidden_api = shuffle_arrays(input_zero_forbidden_api, labels_zero_forbidden_api)
input_one_forbidden_api, labels_one_forbidden_api = shuffle_arrays(input_one_forbidden_api, labels_one_forbidden_api)
input_two_forbidden_api, labels_two_forbidden_api = shuffle_arrays(input_two_forbidden_api, labels_two_forbidden_api)
input_three_forbidden_api, labels_three_forbidden_api = shuffle_arrays(input_three_forbidden_api, labels_three_forbidden_api)

In [ ]:
# shuffle glove data
input_zero_forbidden_glove, labels_zero_forbidden_glove = shuffle_arrays(input_zero_forbidden_glove, labels_zero_forbidden_glove)
input_one_forbidden_glove, labels_one_forbidden_glove = shuffle_arrays(input_one_forbidden_glove, labels_one_forbidden_glove)
input_two_forbidden_glove, labels_two_forbidden_glove = shuffle_arrays(input_two_forbidden_glove, labels_two_forbidden_glove)
input_three_forbidden_glove, labels_three_forbidden_glove = shuffle_arrays(input_three_forbidden_glove, labels_three_forbidden_glove)

# Build Neural Network

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as kb

## Model API Embedding

In [ ]:
def create_api_nn_model():
    model_with_api_embedding = tf.keras.models.Sequential()

    model_with_api_embedding.add(tf.keras.layers.Dense(units=15, activation='linear')) # input layer
    model_with_api_embedding.add(tf.keras.layers.Dense(units=128, activation='relu'))   # hidden layer
    model_with_api_embedding.add(tf.keras.layers.Dense(units=len(labels_zero_forbidden_api[0]), activation='sigmoid')) # output layer

    model_with_api_embedding.compile(optimizer = 'adam', loss = 'binary_crossentropy')
    return model_with_api_embedding

## Model Glove Embedding

In [ ]:
def create_glove_nn_model():
    model_with_glove_embedding = tf.keras.models.Sequential()

    model_with_glove_embedding.add(tf.keras.layers.Dense(units=15, activation='linear')) # input layer
    model_with_glove_embedding.add(tf.keras.layers.Dense(units=128, activation='relu'))   # hidden layer
    model_with_glove_embedding.add(tf.keras.layers.Dense(units=len(labels_zero_forbidden_glove[0]), activation='sigmoid')) # output layer

    model_with_glove_embedding.compile(optimizer = 'adam', loss = 'binary_crossentropy')
    return model_with_glove_embedding

## Test / Train Classifiers

In [ ]:
def get_nn_prediction_result(res_predicted, res_actual, log=False):
    prediction = []

    for i, value in enumerate(res_predicted):
        prediction.append([i, value])
        

    def sortkey(el):
        return el[1]

    prediction.sort(key=sortkey, reverse=True)
    if log:
        print("prediction order:")
        print(prediction)

    prediction_order = [i[0] for i in prediction]

    prediction = None # free ram
    
    indices_words_to_predict = [i for i in range(len(res_actual)) if res_actual[i]==1]

    if log:
        print("indices to predict: "+str(indices_words_to_predict))

    prediction_delays = [max(0, prediction_order.index(w)-len(indices_words_to_predict)+1) for w in indices_words_to_predict]

    if log:
        print("Prediction delays: "+str(prediction_delays))

    number_correct = 0
    for i in prediction_delays:
        if i==0:
            number_correct += 1

    portion_correct = number_correct/len(prediction_delays)
    avg_delay = sum(prediction_delays)/len(prediction_delays)
    
    return portion_correct, avg_delay

In [ ]:
from statistics import median

def get_avg_prediction_delay(predictions_outputs, wanted_outputs):
    delays = []

    for i in range(len(predictions_outputs)):
        delays.append(get_nn_prediction_result(predictions_outputs[i], wanted_outputs[i])[1])

    delays = np.array(delays)
    avg_delay = np.sum(delays)/len(delays)

    return avg_delay, median(delays)

# Perpetual Train and Test

In [ ]:
def perpetual_test_handle_forbidden_words():
    while(True):
        forbidden_words = get_forbidden_words()

        # generate training and test data
        input_zero_forbidden_api, labels_zero_forbidden_api, input_one_forbidden_api, labels_one_forbidden_api, input_two_forbidden_api, labels_two_forbidden_api, input_three_forbidden_api, labels_three_forbidden_api = get_data_for_training(requests, api_embedding, index_of_word_api, forbidden_words)

        input_zero_forbidden_glove, labels_zero_forbidden_glove, input_one_forbidden_glove, labels_one_forbidden_glove, input_two_forbidden_glove, labels_two_forbidden_glove, input_three_forbidden_glove, labels_three_forbidden_glove = get_data_for_training(requests, glove_embedding, index_of_word_glove, forbidden_words)

        # shuffle
        input_zero_forbidden_api, labels_zero_forbidden_api = shuffle_arrays(input_zero_forbidden_api, labels_zero_forbidden_api)
        input_one_forbidden_api, labels_one_forbidden_api = shuffle_arrays(input_one_forbidden_api, labels_one_forbidden_api)
        input_two_forbidden_api, labels_two_forbidden_api = shuffle_arrays(input_two_forbidden_api, labels_two_forbidden_api)
        input_three_forbidden_api, labels_three_forbidden_api = shuffle_arrays(input_three_forbidden_api, labels_three_forbidden_api)

        input_zero_forbidden_glove, labels_zero_forbidden_glove = shuffle_arrays(input_zero_forbidden_glove, labels_zero_forbidden_glove)
        input_one_forbidden_glove, labels_one_forbidden_glove = shuffle_arrays(input_one_forbidden_glove, labels_one_forbidden_glove)
        input_two_forbidden_glove, labels_two_forbidden_glove = shuffle_arrays(input_two_forbidden_glove, labels_two_forbidden_glove)
        input_three_forbidden_glove, labels_three_forbidden_glove = shuffle_arrays(input_three_forbidden_glove, labels_three_forbidden_glove)

        # create model
        model_with_api_embedding = create_api_nn_model()
        model_with_glove_embedding = create_glove_nn_model()

        print("-------------------------------------------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------------------------------------------")
        print()

        print("forbidden input words: "+str(forbidden_words))
        print()
        print("size data without forbidden input (training data) : "+str(len(labels_zero_forbidden_api)))
        print("size data with 1 forbidden word                   : "+str(len(labels_one_forbidden_api)))
        print("size data with 2 forbidden words                  : "+str(len(labels_two_forbidden_api)))
        print("size data with 3 forbidden words                  : "+str(len(labels_three_forbidden_api)))
        print()

        prediction_zero_forbidden_api = model_with_api_embedding.predict(input_zero_forbidden_api)
        prediction_zero_forbidden_glove = model_with_glove_embedding.predict(input_zero_forbidden_glove)

        prediction_one_forbidden_api = model_with_api_embedding.predict(input_one_forbidden_api)
        prediction_one_forbidden_glove = model_with_glove_embedding.predict(input_one_forbidden_glove)

        prediction_two_forbidden_api = model_with_api_embedding.predict(input_two_forbidden_api)
        prediction_two_forbidden_glove = model_with_glove_embedding.predict(input_two_forbidden_glove)

        prediction_three_forbidden_api = model_with_api_embedding.predict(input_three_forbidden_api)
        prediction_three_forbidden_glove = model_with_glove_embedding.predict(input_three_forbidden_glove)

        print("   ---   Result without Training   ---")
        print()
        print("Result avg delay zero forbidden words (training data):")
        avg_delay_zero_forbidden_api, median_delay_zero_forbidden_api = get_avg_prediction_delay(prediction_zero_forbidden_api, labels_zero_forbidden_api)
        print("api: avg = "+str(avg_delay_zero_forbidden_api)+" median = "+str(median_delay_zero_forbidden_api))
        avg_delay_zero_forbidden_glove, median_delay_zero_forbidden_glove = get_avg_prediction_delay(prediction_zero_forbidden_glove, labels_zero_forbidden_glove)
        print("glove: avg = "+str(avg_delay_zero_forbidden_glove)+" median = "+str(median_delay_zero_forbidden_glove))
        print()

        print("Result avg delay one forbidden word:")
        avg_delay_one_forbidden_api, median_delay_one_forbidden_api = get_avg_prediction_delay(prediction_one_forbidden_api, labels_one_forbidden_api)
        print("api: avg = "+str(avg_delay_one_forbidden_api)+" median = "+str(median_delay_one_forbidden_api))
        avg_delay_one_forbidden_glove, median_delay_one_forbidden_glove = get_avg_prediction_delay(prediction_one_forbidden_glove, labels_one_forbidden_glove)
        print("glove: avg = "+str(avg_delay_one_forbidden_glove)+" median = "+str(median_delay_one_forbidden_glove))
        print()

        print("Result avg delay two forbidden words:")
        avg_delay_two_forbidden_api, median_delay_two_forbidden_api = get_avg_prediction_delay(prediction_two_forbidden_api, labels_two_forbidden_api)
        print("api: avg = "+str(avg_delay_two_forbidden_api)+" median = "+str(median_delay_two_forbidden_api))
        avg_delay_two_forbidden_glove, median_delay_two_forbidden_glove = get_avg_prediction_delay(prediction_two_forbidden_glove, labels_two_forbidden_glove)
        print("glove: avg = "+str(avg_delay_two_forbidden_glove)+" median = "+str(median_delay_two_forbidden_glove))
        print()

        print("Result avg delay three forbidden words:")
        avg_delay_three_forbidden_api, median_delay_three_forbidden_api = get_avg_prediction_delay(prediction_three_forbidden_api, labels_three_forbidden_api)
        print("api: avg = "+str(avg_delay_three_forbidden_api)+" median = "+str(median_delay_three_forbidden_api))
        avg_delay_three_forbidden_glove, median_delay_three_forbidden_glove = get_avg_prediction_delay(prediction_three_forbidden_glove, labels_three_forbidden_glove)
        print("glove: avg = "+str(avg_delay_three_forbidden_glove)+" median = "+str(median_delay_three_forbidden_glove))
        print()
        print()

        # train
        model_with_glove_embedding.fit(input_zero_forbidden_glove, labels_zero_forbidden_glove, batch_size = 8, epochs = 20)
        model_with_api_embedding.fit(input_zero_forbidden_api, labels_zero_forbidden_api, batch_size = 8, epochs = 20)

        # predict
        prediction_zero_forbidden_api = model_with_api_embedding.predict(input_zero_forbidden_api)
        prediction_zero_forbidden_glove = model_with_glove_embedding.predict(input_zero_forbidden_glove)

        prediction_one_forbidden_api = model_with_api_embedding.predict(input_one_forbidden_api)
        prediction_one_forbidden_glove = model_with_glove_embedding.predict(input_one_forbidden_glove)

        prediction_two_forbidden_api = model_with_api_embedding.predict(input_two_forbidden_api)
        prediction_two_forbidden_glove = model_with_glove_embedding.predict(input_two_forbidden_glove)

        prediction_three_forbidden_api = model_with_api_embedding.predict(input_three_forbidden_api)
        prediction_three_forbidden_glove = model_with_glove_embedding.predict(input_three_forbidden_glove)


        print()
        print("   ---   Result after 20 Epochs Training   ---")
        print()
        print("Result avg delay zero forbidden words (training data):")
        avg_delay_zero_forbidden_api, median_delay_zero_forbidden_api = get_avg_prediction_delay(prediction_zero_forbidden_api, labels_zero_forbidden_api)
        print("api: avg = "+str(avg_delay_zero_forbidden_api)+" median = "+str(median_delay_zero_forbidden_api))
        avg_delay_zero_forbidden_glove, median_delay_zero_forbidden_glove = get_avg_prediction_delay(prediction_zero_forbidden_glove, labels_zero_forbidden_glove)
        print("glove: avg = "+str(avg_delay_zero_forbidden_glove)+" median = "+str(median_delay_zero_forbidden_glove))
        print()

        print("Result avg delay one forbidden word:")
        avg_delay_one_forbidden_api, median_delay_one_forbidden_api = get_avg_prediction_delay(prediction_one_forbidden_api, labels_one_forbidden_api)
        print("api: avg = "+str(avg_delay_one_forbidden_api)+" median = "+str(median_delay_one_forbidden_api))
        avg_delay_one_forbidden_glove, median_delay_one_forbidden_glove = get_avg_prediction_delay(prediction_one_forbidden_glove, labels_one_forbidden_glove)
        print("glove: avg = "+str(avg_delay_one_forbidden_glove)+" median = "+str(median_delay_one_forbidden_glove))
        print()

        print("Result avg delay two forbidden words:")
        avg_delay_two_forbidden_api, median_delay_two_forbidden_api = get_avg_prediction_delay(prediction_two_forbidden_api, labels_two_forbidden_api)
        print("api: avg = "+str(avg_delay_two_forbidden_api)+" median = "+str(median_delay_two_forbidden_api))
        avg_delay_two_forbidden_glove, median_delay_two_forbidden_glove = get_avg_prediction_delay(prediction_two_forbidden_glove, labels_two_forbidden_glove)
        print("glove: avg = "+str(avg_delay_two_forbidden_glove)+" median = "+str(median_delay_two_forbidden_glove))
        print()

        print("Result avg delay three forbidden words:")
        avg_delay_three_forbidden_api, median_delay_three_forbidden_api = get_avg_prediction_delay(prediction_three_forbidden_api, labels_three_forbidden_api)
        print("api: avg = "+str(avg_delay_three_forbidden_api)+" median = "+str(median_delay_three_forbidden_api))
        avg_delay_three_forbidden_glove, median_delay_three_forbidden_glove = get_avg_prediction_delay(prediction_three_forbidden_glove, labels_three_forbidden_glove)
        print("glove: avg = "+str(avg_delay_three_forbidden_glove)+" median = "+str(median_delay_three_forbidden_glove))
        print()
        print()

In [ ]:
perpetual_test_handle_forbidden_words()